#### 1. Load the data and libraries

In [1]:
import os
from vpirag import *
from dotenv import load_dotenv
from llama_index.llms import OpenAI
from llama_index.llms import AzureOpenAI
from llama_index.embeddings import OpenAIEmbedding

load_dotenv()

True

In [2]:
# Set OpenAI API key
# openai.api_key = os.environ['OPENAI_API_KEY']
# Set Qdrant API key
qdrant_url = os.environ['QDRANT_URL']
qdrant_api_key = os.environ['QDRANT_API_KEY']
qdrant_collection = os.environ['QDRANT_COLLECTION_NAME']
# Set Azure OpenAI API key
openai.api_type = 'azure'
openai.api_key = os.environ['AZURE_OPENAI_KEY']
openai.api_base = os.environ['AZURE_OPENAI_ENDPOINT']
openai.api_version = os.environ['AZURE_OPENAI_API_VERSION']

#### 2. Define the functions

In [3]:
embedding_model = "azureopenai"

if embedding_model == "azureopenai":
    embedding_dim = 1536
if embedding_model == "openai":
    embedding_dim = 1536
elif embedding_model == "cohere":
    embedding_dim = 768

# Note the uninformative document file name, which may be a common scenario in a production setting
loaded_docs = SimpleDirectoryReader(input_dir="./data/blv_data").load_data()

# Set up ServiceContext inlude llm and node_parser
# llm = OpenAI(temperature=0.0, model="gpt-3.5-turbo", max_tokens=2000)

## Using AzureOpenAI
llm = AzureOpenAI(
    temperature=0.0,
    api_type = "azure",
    engine = "vpi35turbo16k",
    api_key = os.environ['AZURE_OPENAI_KEY'],
    api_base = os.environ['AZURE_OPENAI_ENDPOINT'],
    api_version = os.environ['AZURE_OPENAI_API_VERSION'],
)

## Embedding model
embed_model = OpenAIEmbedding(
    deployment_name="vpi-embedding-ada-002",
    api_type = "azure",
    api_key = os.environ['AZURE_OPENAI_KEY'],
    api_base = os.environ['AZURE_OPENAI_ENDPOINT'],
    api_version = os.environ['AZURE_OPENAI_API_VERSION'],
)
# Build index
for loaded_doc in loaded_docs:
    ## Indexing and metadata extraction
    index = metadata_creator(llm=llm, 
                             embed_model=embed_model, 
                             loaded_doc=[loaded_doc],
                             separator="\n\n",
                             chunk_size=1024, 
                             chunk_overlap=128,
                             )
    ## Read documents
    docs = index.docstore.docs
    ## Transfer to Qdrant Point, vector with payload
    qdrant_client = qdrant_collection_def(qdrant_url, qdrant_api_key, qdrant_collection, embedding_dim)
    ## Upload to Qdrant
    qdrant_uploader(qdrant_client, embedding_model, qdrant_collection, docs)


Parsing documents into nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting questions:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting summaries:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Document ID: 3b1d4f20-8ecd-460f-9578-6d1be28c71d0, File Name: bachlongvi_text.docx
Qdrant Upserting Finished!


In [6]:
docs['3b1d4f20-8ecd-460f-9578-6d1be28c71d0'].metadata

{'file_name': 'bachlongvi_text.docx',
 'document_title': 'Dự báo môi trường trầm tích dựa trên mô hình lý thuyết rift 3 pha',
 'questions_this_excerpt_can_answer': '1. Mô hình lý thuyết rift 3 pha được sử dụng để dự báo môi trường trầm tích như thế nào?\n2. Kết quả phân tích ĐTPT theo mô hình lý thuyết rift 3 pha chỉ ra những đặc điểm gì về phân tập đầu tiên và phân tập thứ hai?\n3. Môi trường năng lượng cao như sông suối có liên quan đến phân tập đầu tiên của pha tách giãn không?\n4. Môi trường ven hồ hoặc hồ nông có thể là môi trường trầm tích trong khu vực trũng không?\n5. Trầm tích hình thành trong môi trường hồ nước sâu có những đặc điểm gì đối với phân tập thứ hai của pha tách giãn?',
 'section_summary': 'Trong phần này, nhóm tác giả sử dụng mô hình lý thuyết rift 3 pha để dự báo môi trường trầm tích. Kết quả phân tích cho thấy phân tập đầu tiên của pha tách giãn liên quan đến môi trường năng lượng cao như sông suối và có thể là môi trường ven hồ hoặc hồ nông. Phân tập thứ hai hì